## Proyecto 3 - Modelización

Desarrollado por David Gutiérrez & Natalia Núñez

![Descripción de la imagen](RED_ICFES.png)

In [10]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import math
from pgmpy.models import BayesianModel
from pgmpy.models import BayesianNetwork
from pgmpy.estimators import MaximumLikelihoodEstimator, BayesianEstimator
from pgmpy.inference import VariableElimination
from sklearn.metrics import confusion_matrix
import numpy as np
import os
from pgmpy.sampling import BayesianModelSampling
from pgmpy.factors.discrete import TabularCPD
from sklearn.model_selection import train_test_split


In [5]:

# Crear un DataFrame de ejemplo
df = pd.DataFrame({'Municipio': ['Municipio A', 'Municipio B', 'Municipio A', 'Municipio C', 'Municipio B']})

# Obtener los números únicos para cada municipio
df['Municipio_Numero'] = pd.factorize(df['Municipio'])[0]

# Imprimir el DataFrame resultante
print(df)


     Municipio  Municipio_Numero
0  Municipio A                 0
1  Municipio B                 1
2  Municipio A                 0
3  Municipio C                 2
4  Municipio B                 1


In [6]:
# Leer el archivo CSV

df = pd.read_csv("datos_p3.csv")
print(df.columns)
print(df["FAMI_TIENEINTERNET"].describe())
print(df["FAMI_TIENEINTERNET"].unique())

Index(['PERIODO', 'COLE_AREA_UBICACION', 'COLE_BILINGUE',
       'COLE_DEPTO_UBICACION', 'COLE_JORNADA', 'COLE_MCPIO_UBICACION',
       'ESTU_GENERO', 'FAMI_EDUCACIONMADRE', 'FAMI_EDUCACIONPADRE',
       'FAMI_ESTRATOVIVIENDA', 'FAMI_TIENEINTERNET', 'PUNT_GLOBAL'],
      dtype='object')
count     869322
unique         2
top           Si
freq      541859
Name: FAMI_TIENEINTERNET, dtype: object
['No' 'Si']


In [7]:
#Tenemos que discretizar periodo y puntaje global
#Discretizar cole area ubicacion
df["COLE_AREA_UBICACION_DISCRETO"] = df["COLE_AREA_UBICACION"]
df['COLE_AREA_UBICACION'] = pd.factorize(df['COLE_AREA_UBICACION_DISCRETO'])[0]

#Discretizar cole bilingue
df["COLE_BILINGUE_DISCRETO"] = df["COLE_BILINGUE"]
df['COLE_BILINGUE'] = pd.factorize(df['COLE_BILINGUE_DISCRETO'])[0]

#Discretizar COLE_DEPTO_UBICACION
df["COLE_DEPTO_UBICACION_DISCRETO"] = df["COLE_DEPTO_UBICACION"]
df['COLE_DEPTO_UBICACION'] = pd.factorize(df['COLE_DEPTO_UBICACION_DISCRETO'])[0]

#Discretizar COLE_JORNADA
df["COLE_JORNADA_DISCRETO"] = df["COLE_JORNADA"]
df['COLE_JORNADA'] = pd.factorize(df['COLE_JORNADA_DISCRETO'])[0]

#Discretizar COLE_MCPIO_UBICACION
df["COLE_MCPIO_UBICACION_DISCRETO"] = df["COLE_MCPIO_UBICACION"]
df['COLE_MCPIO_UBICACION'] = pd.factorize(df['COLE_MCPIO_UBICACION_DISCRETO'])[0]

#Discretizar FAMI_EDUCACIONMADRE
df["FAMI_EDUCACIONMADRE_DISCRETO"] = df["FAMI_EDUCACIONMADRE"]
df['FAMI_EDUCACIONMADRE'] = pd.factorize(df['FAMI_EDUCACIONMADRE_DISCRETO'])[0]

#Discretizar FAMI_EDUCACIONPADRE
df["FAMI_EDUCACIONPADRE_DISCRETO"] = df["FAMI_EDUCACIONPADRE"]
df['FAMI_EDUCACIONPADRE'] = pd.factorize(df['FAMI_EDUCACIONPADRE_DISCRETO'])[0]

#Discretizar FAMI_ESTRATOVIVIENDA
df["FAMI_ESTRATOVIVIENDA_DISCRETO"] = df["FAMI_ESTRATOVIVIENDA"]
df['FAMI_ESTRATOVIVIENDA'] = pd.factorize(df['FAMI_ESTRATOVIVIENDA_DISCRETO'])[0]

#Discretizar FAMI_TIENEINTERNET
df["FAMI_TIENEINTERNET_DISCRETO"] = df["FAMI_TIENEINTERNET"]
df['FAMI_TIENEINTERNET'] = pd.factorize(df['FAMI_TIENEINTERNET_DISCRETO'])[0]

#Discretizar puntaje global

df["PUNT_GLOBAL_DISCRETO"] = df["PUNT_GLOBAL"]

df.loc[df["PUNT_GLOBAL_DISCRETO"] < 100, "PUNT_GLOBAL" ] = 1
df.loc[(100 <= df["PUNT_GLOBAL_DISCRETO"]) & (df["PUNT_GLOBAL_DISCRETO"] < 200), "PUNT_GLOBAL" ] = 2
df.loc[(200 <= df["PUNT_GLOBAL_DISCRETO"]) & (df["PUNT_GLOBAL_DISCRETO"] < 300), "PUNT_GLOBAL" ] = 3
df.loc[(300 <= df["PUNT_GLOBAL_DISCRETO"]) & (df["PUNT_GLOBAL_DISCRETO"] < 400), "PUNT_GLOBAL" ] = 4
df.loc[(400 <= df["PUNT_GLOBAL_DISCRETO"]) & (df["PUNT_GLOBAL_DISCRETO"] < 500), "PUNT_GLOBAL" ] = 5
df.PUNT_GLOBAL = df.PUNT_GLOBAL.astype(int)
#No lo toma como numero

#Discretizar periodo
df["PERIODO_DISCRETO"] = df["PERIODO"]

df.loc[(df["PERIODO_DISCRETO"] == 20191), "PERIODO"] = 1
df.loc[(df["PERIODO_DISCRETO"] == 20194), "PERIODO"] = 4
df.PERIODO = df.PERIODO.astype(int)

#Eliminamos columnas sobrantes
df.drop(["PERIODO_DISCRETO", "PUNT_GLOBAL_DISCRETO","COLE_AREA_UBICACION_DISCRETO","COLE_BILINGUE_DISCRETO",
         "COLE_DEPTO_UBICACION_DISCRETO","COLE_JORNADA_DISCRETO","COLE_MCPIO_UBICACION_DISCRETO",
         "FAMI_EDUCACIONMADRE_DISCRETO","FAMI_EDUCACIONPADRE_DISCRETO","FAMI_ESTRATOVIVIENDA_DISCRETO","FAMI_TIENEINTERNET_DISCRETO"], axis=1, inplace=True)
df


,PERIODO,COLE_AREA_UBICACION,COLE_BILINGUE,COLE_DEPTO_UBICACION,COLE_JORNADA,COLE_MCPIO_UBICACION,ESTU_GENERO,FAMI_EDUCACIONMADRE,FAMI_EDUCACIONPADRE,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,PUNT_GLOBAL
0,4,0,0,0,0,0,M,0,0,0,0,3
1,4,0,0,1,0,1,F,1,1,0,0,4
2,4,0,0,1,0,1,F,1,1,0,0,4
3,4,0,0,2,1,2,M,2,2,0,1,3
4,4,0,0,2,1,2,M,2,2,0,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...
869317,4,0,0,9,3,490,F,6,9,1,0,3
869318,4,0,0,19,3,629,M,0,8,0,0,3
869319,4,0,0,19,3,629,M,0,8,0,0,3
869320,4,1,0,17,3,383,M,3,9,1,0,3


In [8]:
#Creamos modelo
model = BayesianNetwork(
    [ ("FAMI_TIENEINTERNET", "PUNT_GLOBAL"),
     ("COLE_BILINGUE", "PUNT_GLOBAL"),
     ("FAMI_ESTRATOVIVIENDA", "FAMI_TIENEINTERNET"),
     ("PERIODO", "COLE_BILINGUE"),
     ("COLE_JORNADA", "COLE_BILINGUE"),
     ("FAMI_EDUCACIONMADRE", "FAMI_ESTRATOVIVIENDA"),
     ("FAMI_EDUCACIONPADRE", "FAMI_ESTRATOVIVIENDA"),
     ("COLE_MCPIO_UBICACION", "COLE_BILINGUE"),     
     ("COLE_DEPTO_UBICACION", "COLE_MCPIO_UBICACION"),
     ("COLE_AREA_UBICACION", "COLE_DEPTO_UBICACION"),
     ("COLE_AREA_UBICACION", "FAMI_ESTRATOVIVIENDA"),
     ("FAMI_ESTRATOVIVIENDA", "COLE_BILINGUE"),

    ]
)

In [9]:
#MAXIMUM LIKELIHOOD
model.fit(
    data=df,
    estimator=MaximumLikelihoodEstimator
)

for i in model.nodes():
    print(i)
    print(model.get_cpds(i))


FAMI_TIENEINTERNET
+-----------------------+-----+--------------------------+
| FAMI_ESTRATOVIVIENDA  | ... | FAMI_ESTRATOVIVIENDA(17) |
+-----------------------+-----+--------------------------+
| FAMI_TIENEINTERNET(0) | ... | 0.0                      |
+-----------------------+-----+--------------------------+
| FAMI_TIENEINTERNET(1) | ... | 1.0                      |
+-----------------------+-----+--------------------------+
PUNT_GLOBAL
+--------------------+-----+-----------------------+
| COLE_BILINGUE      | ... | COLE_BILINGUE(1)      |
+--------------------+-----+-----------------------+
| FAMI_TIENEINTERNET | ... | FAMI_TIENEINTERNET(1) |
+--------------------+-----+-----------------------+
| PUNT_GLOBAL(1)     | ... | 0.0                   |
+--------------------+-----+-----------------------+
| PUNT_GLOBAL(2)     | ... | 0.08584091439234896   |
+--------------------+-----+-----------------------+
| PUNT_GLOBAL(3)     | ... | 0.37030557499416844   |
+--------------------+----

In [7]:
def inference(evidence):
    infer=VariableElimination(model)
    prob=infer.query(variables=['PUNT_GLOBAL'], evidence=evidence)
    return prob.values
print(inference({
        'PERIODO':1,
        'COLE_MCPIO_UBICACION':58}))

[1.11852754e-04 1.78487573e-01 5.50983276e-01 2.62425417e-01
 7.99188100e-03]


In [12]:
#Método evaluativo modelo

df.isna().sum()
train_data = df.iloc[:695458] #Partimos que el 80% sean de entrenamiento
test_data =df.iloc[695458:] #Partimos que el 20% restante sean de testeo

model_entrenamiento = BayesianModel(
    [ ("FAMI_TIENEINTERNET", "PUNT_GLOBAL"),
     ("COLE_BILINGUE", "PUNT_GLOBAL"),
     ("FAMI_ESTRATOVIVIENDA", "FAMI_TIENEINTERNET"),
     ("PERIODO", "COLE_BILINGUE"),
     ("COLE_JORNADA", "COLE_BILINGUE"),
     ("FAMI_EDUCACIONMADRE", "FAMI_ESTRATOVIVIENDA"),
     ("FAMI_EDUCACIONPADRE", "FAMI_ESTRATOVIVIENDA"),
     ("COLE_MCPIO_UBICACION", "COLE_BILINGUE"),     
     ("COLE_DEPTO_UBICACION", "COLE_MCPIO_UBICACION"),
     ("COLE_AREA_UBICACION", "COLE_DEPTO_UBICACION"),
     ("COLE_AREA_UBICACION", "FAMI_ESTRATOVIVIENDA"),
     ("FAMI_ESTRATOVIVIENDA", "COLE_BILINGUE"),

    ]
)

model_entrenamiento.fit(train_data, estimator=MaximumLikelihoodEstimator)

# Creamos VariableElimination para los 2 modelos
y_pred_test = []
infer_train = VariableElimination(model_entrenamiento)


for i in range(len(test_data)):
    queryt = infer_train.query(variables=['PUNT_GLOBAL'], evidence={'PERIODO': test_data.iloc[i,0], 'COLE_AREA_UBICACION': test_data.iloc[i,1],'COLE_BILINGUE': test_data.iloc[i,2],'COLE_DEPTO_UBICACION': test_data.iloc[i,3],
                                                                    'COLE_JORNADA': test_data.iloc[i,4],'COLE_MCPIO_UBICACION': test_data.iloc[i,5],'FAMI_EDUCACIONMADRE': test_data.iloc[i,7], 'FAMI_EDUCACIONPADRE': test_data.iloc[i,8],
                                                                     'FAMI_EDUCACIONPADRE': test_data.iloc[i,8], 'FAMI_ESTRATOVIVIENDA': test_data.iloc[i,9], 'FAMI_TIENEINTERNET': test_data.iloc[i,10] })   
    prob2 = queryt.values
    
    
    if prob2[1]>=0.5:

        y_pred_test.append(1)
    else:
        y_pred_test.append(0)  # 1 es el índice de la etiqueta positiva

C:\Users\natty\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pgmpy\models\BayesianModel.py:8: FutureWarning: BayesianModel has been renamed to BayesianNetwork. Please use BayesianNetwork class, BayesianModel will be removed in future.
  warnings.warn(
C:\Users\natty\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pgmpy\models\BayesianModel.py:8: FutureWarning: BayesianModel has been renamed to BayesianNetwork. Please use BayesianNetwork class, BayesianModel will be removed in future.
  warnings.warn(
C:\Users\natty\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pgmpy\factors\discrete\DiscreteFactor.py:478: RuntimeWarning: invalid value encountered in divide
  phi.values = phi.values / phi.values.sum()


KeyError: 17